In [17]:
import os
import shutil
import wandb
import torch
%load_ext autoreload

# Step 1: Clear Environment Variables
os.environ.pop('WANDB_API_KEY', None)

# Step 2: Clear Wandb Config Directory
wandb_config_dir = os.path.expanduser("~/.config/wandb")
if os.path.exists(wandb_config_dir):
    shutil.rmtree(wandb_config_dir)

import os
print("CUDA_VISIBLE_DEVICES:", os.environ.get('CUDA_VISIBLE_DEVICES'))

# Try to force PyTorch to see the GPU
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

!nvidia-smi

print("PyTorch version:", torch.__version__)
print("Is CUDA available:", torch.cuda.is_available())
print("CUDA version:", torch.version.cuda if torch.cuda.is_available() else "N/A")

# Print information about available GPUs
if torch.cuda.is_available():
    print("Number of GPUs:", torch.cuda.device_count())
    for i in range(torch.cuda.device_count()):
        print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
else:
    print("No GPU available")

# Check if CUDA is initialized
print("Is CUDA initialized:", torch.cuda.is_initialized())

# If CUDA is available but not initialized, try to initialize it
if torch.cuda.is_available() and not torch.cuda.is_initialized():
    try:
        torch.cuda.init()
        print("CUDA initialized successfully")
    except Exception as e:
        print(f"Failed to initialize CUDA: {e}")

# Set the default tensor type to cuda if available
if torch.cuda.is_available():
    torch.set_default_tensor_type(torch.cuda.FloatTensor)
    print("Default tensor type set to CUDA")
else:
    print("Using CPU tensors")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
CUDA_VISIBLE_DEVICES: 0
Mon Jul 15 18:15:01 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.183.06             Driver Version: 535.183.06   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A10G                    Off | 00000000:00:1E.0 Off |                    0 |
|  0%   39C    P0              64W / 300W |    256MiB / 23028MiB |      0%      Default |
|                                         |   

In [10]:
import sys
sys.path.append('/teamspace/studios/this_studio/wildfire-ignition-generator/src')
import os
os.chdir('/teamspace/studios/this_studio/wildfire-ignition-generator')


In [16]:
!pwd


/teamspace/studios/this_studio/wildfire-ignition-generator


In [18]:
from src.data.geo_data_handler import GeoDataHandler
from src.utils.tile_manager import TileManager
from src.models.cnf_utils import CNFUtils
from src.models.normalizing_flow import ConditionalNormalizingFlow


def reload_mods():
    """
    Function to toggle the automatic reloading of modules in an IPython environment.
    It enables the autoreload feature which reloads modules automatically before executing code.
    """
    # Enable autoreload mode 2, which reloads all modules (except those excluded by %aimport)
    %autoreload 2
    print("Auto-reloading of all modules is enabled.")

# Usage
reload_mods()


def process_ignitions_csv(self, csv_path, output_path, date, percentile=95):
    # Load the ignitions CSV
    ignitions_gdf = self.load_ignitions_csv(csv_path)
    
    # Prepare the data for prediction
    X, merged_df = self.prepare_ignitions_for_prediction(ignitions_gdf, date)
    
    # Predict fire sizes
    final_df = self.predict_fire_sizes(X, merged_df)
    
    # Calculate the stop size based on the percentile prediction
    final_df['stop_at_size'] = final_df[f'{percentile}th_percentile']
    
    # Add the stop size to the original dataframe
    ignitions_gdf['stop_at_size'] = final_df['stop_at_size']
    
    # Save the updated dataframe
    ignitions_gdf.to_csv(output_path, index=False)
    
    return ignitions_gdf

Auto-reloading of all modules is enabled.


# Operational

In [19]:
# Initialize the WildfireInferenceEngine
config = {
    'data_dir': "wildfire-ignition-generator/data/wildfire_indicators/raw_netcdfs",
    'wui_data_path': 'path_to_wui_proximity.tif',
    'model_dir': 'wildfire-ignition-generator/data/artifacts/run-n6xf1a04-history:v0/local',
    'context_size': 75,
    'latent_size': 1,
    'num_flow_layers': 4,
    'hidden_units': 128,
    'hidden_layers': 3,
    'pyrome_shapefile_path': 'wildfire-ignition-generator/data/pyrome_shp/Pyromes_CONUS_20200206.shp'
}

engine = WildfireInferenceEngine(config)

# Load the necessary data
engine.load_data(2023)  # Assuming the ignitions are for 2023



NameError: name 'WildfireInferenceEngine' is not defined

In [ ]:
# Process each ignition CSV file
csv_files = ['path_to_tile1.csv', 'path_to_tile2.csv', 'path_to_tile3.csv']
output_files = ['path_to_output_tile1.csv', 'path_to_output_tile2.csv', 'path_to_output_tile3.csv']

for input_csv, output_csv in zip(csv_files, output_files):
    date = pd.to_datetime('2023-07-01')  # Set the appropriate date
    processed_df = engine.process_ignitions_csv(input_csv, output_csv, date)
    print(f"Processed {input_csv}. Output saved to {output_csv}")
    print(processed_df.head())  # Display the first few rows of the processed dataframe